In [ ]:
from newspaper import fulltext, Article
from datetime import date, timedelta
import requests
import pandas as pd
from dateutil import parser
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Web Scraper 

Scrapes all of the links for a company from 2015 - Present.

In [ ]:
#Keep in main notebook for easier error handling. 

def fetch_articles(company):
    ChromeOptions = webdriver.ChromeOptions()
    ChromeOptions.add_argument('--incognito')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)
    time.sleep(2)
    news_look = "https://www.newslookup.com"
    time.sleep(1)
    driver.get(news_look)
    time.sleep(3)
    
    driver.find_element_by_css_selector('#lookup').send_keys(company)
    
    #search button
    driver.find_element_by_css_selector('#form-group > div > span > button').click()
    
    #time period
    driver.find_element_by_css_selector('#timeperiod').click()
    
    #2020--------------------------------------------------
    driver.find_element_by_css_selector('#tp_-720').click()
    
    #scroll down
    for i in range(1000):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2020.csv'.format(company))
    
    #2019-------------------------------------------------------
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
    driver.find_element_by_css_selector('#timeperiod').click()
    
    driver.find_element_by_css_selector('#tp_Y2019').click()
    
    #scroll down
    for i in range(10):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2019.csv'.format(company))
    
    #2018-------------------------------------------------------
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
    driver.find_element_by_css_selector('#timeperiod').click()
    
    driver.find_element_by_css_selector('#tp_Y2018').click()
    
    #scroll down
    for i in range(1000):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2018.csv'.format(company))
    
    #2017-------------------------------------------------------
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
    driver.find_element_by_css_selector('#timeperiod').click()
    
    driver.find_element_by_css_selector('#tp_Y2017').click()
    
    #scroll down
    for i in range(10):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2017.csv'.format(company))
    
    #2016-------------------------------------------------------
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
    driver.find_element_by_css_selector('#timeperiod').click()
    
    driver.find_element_by_css_selector('#tp_Y2016').click()
    
    #scroll down
    for i in range(1000):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2016.csv'.format(company))
    
    #2015-------------------------------------------------------
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
    driver.find_element_by_css_selector('#timeperiod').click()
    
    driver.find_element_by_css_selector('#tp_Y2015').click()
    
    #scroll down
    for i in range(1000):
        try:
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except:
            continue
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    final_df = pd.DataFrame(columns= ['update', 'date', 'source', 'author', 'fulltext', 'summary'])
    
    
    counter = 0
    for link, date in pano:
        try:
            article= Article(link)
            article.download()
            article.parse()
            text = article.text
            published = article.publish_date
            auth = article.authors
            source = link
            summ = article.summary
            titl = article.title
            final_df = final_df.append({'update':published, 'date':date, 'source':link, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}, ignore_index=True)
            counter += 1
            print(counter)
            final_df = final_df.drop_duplicates(subset='fulltext')
            print(final_df.shape)
        except:
            continue
    final_df = final_df.drop_duplicates(subset='fulltext')       
    final_df.to_csv('yearly_articles/{}2015.csv'.format(company))
    
    driver.close()

In [ ]:
fetch_articles('Nike')